In [1]:
from glob import glob

In [3]:
files = glob("../../data/text files/*www.stevens.edu off-campus-employment*")
files

['../../data/text files\\https_  www.stevens.edu off-campus-employment isss-f1-students cap-gap-extension.txt',
 '../../data/text files\\https_  www.stevens.edu off-campus-employment isss-f1-students cpt-work-authorization.txt',
 '../../data/text files\\https_  www.stevens.edu off-campus-employment isss-f1-students opt-work-authorization.txt',
 '../../data/text files\\https_  www.stevens.edu off-campus-employment isss-f1-students other-off-campus-employment.txt',
 '../../data/text files\\https_  www.stevens.edu off-campus-employment isss-f1-students stem-extension.txt',
 '../../data/text files\\https_  www.stevens.edu off-campus-employment-0.txt']

In [4]:
out = []

for file in files:
    with open(file, "r") as fp:
        out.append(fp.read())

In [5]:
out

['Students with pending or approved H-1B petitions are permitted to extend their F-1 OPT/STEM OPT work authorization from the end of the OPT/STEM OPT period through September 30. This period, known as the Cap-Gap, only applies to those whose OPT/STEM OPT period ends prior to September 30 and to those with H-1B petitions filed with a request for a change in immigration status, not consular processing. This regulation enables those on OPT/STEM OPT to continue their employment until their H-1B takes effect on October 1.\nPlease note that ISSS is unable to advise students on H-1B specific related matters, and students should always consult with an immigration attorney for any inquiries they might have regarding their H-1B status or their change to H-1B status. ISSS is unable to provide updates to students regarding the H-1B or Cap-Gap information on their SEVIS Records. Student\'s employers or immigration attorneys will always have the most up to date and accurate information surrounding a

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [7]:
model_kwargs = {
    "trust_remote_code": True,
    # "device": "cpu"
    }
encode_kwargs={"normalize_embeddings": True}
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [8]:
docs = [
    Document(page_content=text)
    for text in out
]
vector = Chroma.from_documents(docs, embeddings, collection_name="off_campus_employment", persist_directory="../data/chromadb")